In [ ]:
from models.data_augmentation import selecting_images_preprocessing, sliding_crop \
    , sliding_crop_and_select_one, custom_v3, enhance_image

from models.seresnext50 import build_seresnext50_unet

import tensorflow as tf
from glob import glob
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
IMG_H = 128
IMG_W = 128
IMG_C = 3  ## Change this to 1 for grayscale.
input_shape = (IMG_H, IMG_W, IMG_C)

In [ ]:
def prep_stage(x, train=True):
    beta_contrast = 0.1
    # enchance the brightness
    x = enhance_image(x, beta_contrast)
    # if train:
        # x = enhance_image(x, beta_contrast)
        # x = tfa.image.equalize(x)
        # x = custom_v3(x)
    # else: 
        # x = enhance_image(x, beta_contrast)
        # x = tfa.image.equalize(x)
        # x = custom_v3(x)      
    return x

def post_stage(x):
    
    x = tf.image.resize(x, (IMG_H, IMG_W))
    # x = tf.image.resize_with_crop_or_pad(x, IMG_H, IMG_W)
    # normalize to the range -1,1
    # x = tf.cast(x, tf.float32)
    # x = (x - 127.5) / 127.5
    # normalize to the range 0-1
    # img /= 255.0
    return x

def extraction(image, label):
    # This function will shrink the Omniglot images to the desired size,
    # scale pixel values and convert the RGB image to grayscale
    img = tf.io.read_file(image)
    img = tf.io.decode_png(img, channels=IMG_C)
    # print(image, label)
    # img = cv2.imread(image)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = prep_stage(img, True)
    
    # img = sliding_crop_and_select_one(img)
    # img = post_stage(img)

    return img, label

In [ ]:
test_data_path = "data/mura_april/test_data_v2"
g_model_inner = build_seresnext50_unet(input_shape, IMG_H, IMG_C)
g_filepath = "saved_model/0507/128_rgb_mura_sobel_xy_64_v3_0_seresnext50_20_shots_mura_detection_2000_g_model.h5"

In [ ]:
g_model_inner.load_weights(g_filepath)

normal_image = glob(test_data_path+"/normal/*.png")[0]
defect_image = glob(test_data_path+"/defect/*.png")[0]
paths = {
    "normal": normal_image,
    "defect": defect_image,
}

In [ ]:
for i, v in paths.items():
    print(i,v)

    rows = 1
    cols = 3
    axes=[]
    fig = plt.figure()

    img = tf.io.read_file(v)
    img = tf.io.decode_png(img, channels=IMG_C)
    
    # img = np.clip(img.numpy(), 0, 1)
    
    axes.append( fig.add_subplot(rows, cols, 1) )
    axes[-1].set_title('_original_') 
    plt.imshow(img.numpy().astype(np.uint8))
    plt.axis('off')

    
    img = sliding_crop_and_select_one(img)
    img = post_stage(img)
    
    axes.append( fig.add_subplot(rows, cols, 2) )
    axes[-1].set_title('_preprocessing_') 
    plt.imshow(img.numpy().astype(np.uint8), alpha=1.0)
    plt.axis('off')
    # print(img)
    img = (img - 127.5) / 127.5
    # print(img)
    image = tf.reshape(img, (-1, IMG_H, IMG_W, IMG_C))
    reconstructed_images = g_model_inner.predict(image)
    reconstructed_images = tf.reshape(reconstructed_images, (IMG_H, IMG_W, IMG_C))
    reconstructed_images = reconstructed_images * 127 + 127
     

    # reconstructed_images = np.clip(reconstructed_images.numpy(), 0, 1)
    
    axes.append( fig.add_subplot(rows, cols, 3) )
    axes[-1].set_title('_reconstructed_')
    plt.imshow(reconstructed_images.numpy().astype(np.uint8))
    plt.axis('off')

In [ ]:
import cv2

In [ ]:
def convert_sobel(inputFile, outputFile, x=1, y=1, ks=3, depth=cv2.CV_32F):
    img = cv2.imread(inputFile)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = cv2.Sobel(img, cv2.CV_16U, x, y,ksize=ks)
    img = cv2.Sobel(img, depth, x, y,ksize=ks)
    # img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    return img

In [ ]:
img = cv2.imread(normal_image)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img.astype(np.uint8))

In [ ]:
img = cv2.Sobel(img, cv2.CV_64F, 1, 1,ksize=3)
plt.imshow(img.astype(np.uint8))
# img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite("test4.png", img)